In [18]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import sklearn.model_selection as model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingRegressor
from sklearn.calibration import CalibratedClassifierCV
CCV=CalibratedClassifierCV
from sklearn.metrics import accuracy_score

In [19]:

#IMPORT THE CSV WITH NFL DATA. THERE ARE JUST A FEW THINGS TO DO TO MAKE IT READY TO MODEL



nfl=pd.read_csv('nfl-model-ready.csv')

nfl['Date']=pd.to_datetime(nfl['Date'])
nfl.drop('Unnamed: 0',axis=1,inplace=True)
nfl_my_copy=nfl.copy()
nfl.drop(['fav','dog','team_favorite_id',
          'spread_favorite','month_y','team_home','team_away'],axis=1,inplace=True)


#USING A LABEL ENCODER, WE CAN TURN THESE STRING VALUES INTO NUMERICAL VALUES SO THEY CAN BE ACCOUNTED FOR IN THE MODELING PROCESS

nfl['home']=le.fit_transform(nfl['home'])
nfl['away']=le.fit_transform(nfl['away'])
nfl['stadium']=le.fit_transform(nfl['stadium'])
nfl['fav_dup']=le.fit_transform(nfl['fav_dup'])
nfl['dog_dup']=le.fit_transform(nfl['dog_dup'])



In [20]:
#CLEANING THE REST FEATURE

home_r=[]
away_r=[]
for i in nfl['home_rest']:
    i=i[:-24]
    home_r.append(i)
for i in nfl['away_rest']:
    i=i[:-24]
    away_r.append(i)


nfl['home_rest']=home_r
nfl['home_rest']=nfl['home_rest'].astype(int)

nfl['away_rest']=home_r
nfl['away_rest']=nfl['away_rest'].astype(int)

#THE HOME TEAM COVERS IF...
nfl['home_covers']=(((nfl['home_favorite']==1)&(nfl['score_away']-nfl['score_home']<=nfl['Open']))|((nfl['away_favorite']==1)&(nfl['score_home']-nfl['score_away']>=nfl['Open'])))
nfl['home_covers1']=(((nfl['home_favorite_dup']==1)&(nfl['score_away']-nfl['score_home']<=nfl['Open']))|((nfl['away_favorite_dup']==1)&(nfl['score_home']-nfl['score_away']>=nfl['Open'])))

#IF NEITHER TEAM IS FAVORED, THE HOME TEAM COVERS IF THEY WIN
i=0
for i in range(i,len(nfl)):
    if ((nfl.loc[i,'home_favorite_dup']==0)&(nfl.loc[i,'away_favorite_dup']==0)):
        
        if nfl.loc[i,'score_home']-nfl.loc[i,'score_away']>=nfl.loc[i,'Open']:
                nfl.loc[i,'home_covers1']=1
                
        elif nfl.loc[i,'score_home']-nfl.loc[i,'score_away']<nfl.loc[i,'Open']:
            nfl.loc[i,'home_covers1']=0

nfl.rename(columns={'home_covers':'spread_result','home_covers1':'spread_result1'},inplace=True)
nfl['spread_result']=nfl['spread_result'].astype(int)
nfl['spread_result1']=nfl['spread_result1'].astype(int)



In [21]:
#SPLITTING TRAINING AND TESTING DATA FOR DATA INCLUDING THE 2017 SEASON
#WE WILL USE 80% FOR TRAINING AND 20% FOR TESTING
#WE WILL TRAIN USING 5 FOLD CROSS VALIDATION AND USE GRID SEARCH TO TRY AND FIND THE OPTIMAL MODEL
#DATA FOR THE 2018 SEASON IS THE HOLDOUT SET. THE FINAL MODELS ARE JUST USING IT TO PREDICT, NOT TRAIN, AS TO AVOID ANY DATA LEAKAGE


train=nfl.drop(['home_favorite','away_favorite','Date','move','Open','Close','result','score_home','score_away','over_under','spread_result','spread_result1','Week'],axis=1)
train=train[train['schedule_season']<2018]
target=nfl[nfl['schedule_season']<2018]
target=target[['Close']]

#AS YOU CAN SEE DIRECTLY ABOVE, OUR TARGET IS THE CLOSING LINE
#OUR GOAL IS TO ESTIMATE WHAT THE GAMBLING LINE WILL CLOSE AT, COMPARED TO WHAT IT OPENS AT
#IF OUR PROJECTED CLOSING LINE IS A CERTAIN DISTANCE AWAY FROM THE OPENING LINE, 
#THEN WE WILL WANT TO MAKE A BET ON THE OPENING LINE TO OBTAIN "LINE VALUE"

X_train, X_test, y_train, y_test = train_test_split(train,target,test_size=0.2)



#IN THE NEXT FEW CELLS WE WILL TRAIN VARIOUS TYPES OF MODELS

#TO GET POTENTIALLY BETTER RESULTS, YOU CAN TRY SCALING THE DATA AND USING DIFFERENT TYPES OF FEATURE SELECTION


In [108]:
#xgbc_params={'max_depth':[2,3,4,5,6],'min_child_weight':[5,8,10,12,15,20],'subsample':[.5,.6,.7,.8,.9],'colsample_bytree':[.6,.7,.8,.9],'eta':[.05,.001,.005]}
#xgbc_params={'max_depth':[5,6,7,8],'min_child_weight':[3,4,5,6],'subsample':[.7,.8,.9],'colsample_bytree':[.7,.8,.9],'eta':[.05,.07]}
#xgbc_params={'max_depth':[7],'min_child_weight':[4],'subsample':[.8],'colsample_bytree':[.8],'eta':[.05]}
grid=GridSearchCV(estimator=xgb.XGBRegressor(),param_grid=xgbc_params,cv=5,verbose=2)
grid.fit(X_train,y_train)
grid.best_score_, grid.best_params_

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
[CV] colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7, total=   0.2s
[CV] colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7, total=   0.2s
[CV] colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7 
[CV]  colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7, total=   0.2s
[CV] colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7 
[CV]  colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7, total=   0.2s
[CV] colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7 
[CV]  colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.7, total=   0.3s
[CV] colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.8 
[CV]  colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.8, total=   0.2s
[CV] colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subsample=0.8 
[CV]  colsample_bytree=0.7, eta=0.05, max_depth=5, min_child_weight=3, subs

...

[CV]  colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.7, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.7 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.7, total=   0.3s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.7 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.7, total=   0.3s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.7 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.7, total=   0.3s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.8 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.8, total=   0.3s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subsample=0.8 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=6, min_child_weight=6, subs

[CV]  colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.7, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.7 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.7, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.7 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.7, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.7 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.7, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.8, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=7, min_child_weight=5, subs

[CV]  colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.7, total=   0.6s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.7 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.7, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.7 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.7, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.7 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.7, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.8 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.8, total=   0.4s
[CV] colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subsample=0.8 
[CV]  colsample_bytree=0.9, eta=0.07, max_depth=8, min_child_weight=4, subs

[Parallel(n_jobs=1)]: Done 1440 out of 1440 | elapsed:  9.1min finished


(0.6339043340772498,
 {'colsample_bytree': 0.8,
  'eta': 0.05,
  'max_depth': 7,
  'min_child_weight': 4,
  'subsample': 0.8})

In [109]:
knn_params={'n_neighbors':[10,20,30,40,50]}
grid=GridSearchCV(estimator=KNeighborsRegressor(),param_grid=knn_params,cv=5,verbose=2)
grid.fit(X_train,y_train)
grid.best_score_, grid.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_neighbors=10 ..................................................
[CV] ................................... n_neighbors=10, total=   0.1s
[CV] n_neighbors=10 ..................................................
[CV] ................................... n_neighbors=10, total=   0.0s
[CV] n_neighbors=10 ..................................................
[CV] ................................... n_neighbors=10, total=   0.0s
[CV] n_neighbors=10 ..................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ................................... n_neighbors=10, total=   0.0s
[CV] n_neighbors=10 ..................................................
[CV] ................................... n_neighbors=10, total=   0.0s
[CV] n_neighbors=20 ..................................................
[CV] ................................... n_neighbors=20, total=   0.0s
[CV] n_neighbors=20 ..................................................
[CV] ................................... n_neighbors=20, total=   0.0s
[CV] n_neighbors=20 ..................................................
[CV] ................................... n_neighbors=20, total=   0.0s
[CV] n_neighbors=20 ..................................................
[CV] ................................... n_neighbors=20, total=   0.0s
[CV] n_neighbors=20 ..................................................
[CV] ................................... n_neighbors=20, total=   0.0s
[CV] n_neighbors=30 ..................................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.9s finished


(0.5387662794252813, {'n_neighbors': 20})

In [107]:
#dtc_params={'min_samples_leaf':[1,2,3,4,5],'max_depth':[2,3,4,5,6],'criterion':['mse'],'min_samples_split':[2,3,4,5,6]}
#dtc_params={'min_samples_leaf':[4,5,6,7],'max_depth':[3,4,5],'criterion':['mse'],'min_samples_split':[4]}
dtc_params={'min_samples_leaf':[6],'max_depth':[5],'criterion':['mse'],'min_samples_split':[4]}
grid=GridSearchCV(estimator=DecisionTreeRegressor(),param_grid=dtc_params,cv=5,verbose=2)
grid.fit(X_train,y_train)
grid.best_score_, grid.best_params_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4 
[CV]  criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4 
[CV]  criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4 
[CV]  criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4 
[CV]  criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4 
[CV]  criterion=mse, max_depth=3, min_samples_leaf=4, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=3, min_samples_leaf=5, min_samples_split=4 
[CV]  criterion=mse, max_depth=3, min_samples_leaf

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  criterion=mse, max_depth=3, min_samples_leaf=7, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=3, min_samples_leaf=7, min_samples_split=4 
[CV]  criterion=mse, max_depth=3, min_samples_leaf=7, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=3, min_samples_leaf=7, min_samples_split=4 
[CV]  criterion=mse, max_depth=3, min_samples_leaf=7, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=3, min_samples_leaf=7, min_samples_split=4 
[CV]  criterion=mse, max_depth=3, min_samples_leaf=7, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=4, min_samples_leaf=4, min_samples_split=4 
[CV]  criterion=mse, max_depth=4, min_samples_leaf=4, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=4, min_samples_leaf=4, min_samples_split=4 
[CV]  criterion=mse, max_depth=4, min_samples_leaf=4, min_samples_split=4, total=   0.0s
[CV] criterion=mse, max_depth=4, min_samples_leaf=4, min_samples_split=4 
[CV]  criterion=mse, m

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.7s finished


(0.534809368846557,
 {'criterion': 'mse',
  'max_depth': 5,
  'min_samples_leaf': 6,
  'min_samples_split': 4})

In [110]:
#gbr_params={'max_depth':[2,3,4,5,6],'min_samples_leaf':[3,4,5,6,7,8,9],'subsample':[.2,.3,.4,.5,.6,.7],'min_samples_split':[3,4,5,6,7,8],'learning_rate':[.01,.05,.001,.005]}
#gbr_params={'max_depth':[6,7,8],'min_samples_leaf':[7],'subsample':[.6],'min_samples_split':[8,9,10],'learning_rate':[.03,.05,.07]}
#gbr_params={'max_depth':[7],'min_samples_leaf':[7],'subsample':[.6],'min_samples_split':[9],'learning_rate':[.05]}
#grid=GridSearchCV(estimator=GradientBoostingRegressor(),param_grid=gbr_params,cv=5,verbose=2)
#grid.fit(X_train,y_train)
#grid.best_score_, grid.best_params_

In [22]:

#HERE WE TRAIN EACH MODEL INDIVIDUALLY ALONG WITH THE VOTING MODEL. 
#YOU'LL ONLY SEE THE RESULTS FOR THE VOTER BUT YOU CAN UNCOMMENT RESULTS FOR THE OTHER MODELS



from sklearn.ensemble import VotingRegressor
target['Close']=target['Close'].astype(int)

xgbc=xgb.XGBRegressor(max_depth=7,min_child_weight=4,subsample=.8,colsample_bytree=.8,eta=.05)
gnb=GaussianNB()
logr=LogisticRegression(solver='liblinear')
dtc=DecisionTreeRegressor(min_samples_leaf=6,max_depth=5,criterion='mse',min_samples_split=4,)
knn=KNeighborsRegressor(n_neighbors=32)
gbr=GradientBoostingRegressor(max_depth=7,min_samples_leaf=7,subsample=.6,min_samples_split=9,learning_rate=.05)


model=VotingRegressor([('xgbc',xgbc),('gbr',gbr),('dtc',dtc),('knn',knn)])

model.fit(train,target)


model1=gnb
model1.fit(train,target)
                                                                         

model2=logr
model2.fit(train,target)                                                


model3=knn
model3.fit(train,target)


model4=gbr
model4.fit(train,target)


model5=xgbc
model5.fit(train,target)



XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.05, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints=None,
             learning_rate=0.0500000007, max_delta_step=0, max_depth=7,
             min_child_weight=4, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8, tree_method=None,
             validate_parameters=False, verbosity=None)

In [23]:
#GETTING JUST THE 2018 DATA THAT WE WILL PREDICT OUTCOMES WITH USING THE MODELS IN JUST A BIT

new_train=nfl.drop(['home_favorite','away_favorite','Date','move','Open','Close','result','score_home','score_away','over_under','spread_result','spread_result1','Week'],axis=1)
new_train=new_train[new_train['schedule_season']>=2018]


#SETTING UP A DATA FRAME THAT WE WILL USE TO CALCULATE/DISPLAY OUR WIN %
Test=nfl_my_copy[['Date','schedule_week','score_home','score_away','home','away','fav','dog','home_favorite_dup','away_favorite_dup','Open','Close']][nfl_my_copy['schedule_season']>=2018]
Test['spread_result']=nfl['spread_result1']
Test['Pick']=np.NaN
Test['Pick'][Test['Open']==0]='PICK'


#HAVE THE MODELS PREDICT ON THE 2018 DATA
voter=model.predict(new_train)
gbrpred=model4.predict(new_train)
xgbpred=model5.predict(new_train)

#ADDING THE MODELS PREDICTIONS TO THE DATASET
Test['Voter_preds']=voter
Test['gbr_preds']=gbrpred
Test['xgb_preds']=xgbpred
Test['spread_result'][Test['spread_result']==1]=Test['home']
Test['spread_result'][Test['spread_result']==0]=Test['away']
Test['Vote_guess']=abs(Test['Open']-Test['Voter_preds'])
#Test['gbr_guess']=abs(Test['Open']-Test['gbr_preds'])
#Test['xgb_guess']=abs(Test['Open']-Test['xgb_preds'])

#TELLS US WHICH TEAM OUR PREDICTIONS ARE FAVORING
Test['Vote_favor']=np.NaN
Test['Vote_favor'][Test['Voter_preds']<Test['Open']]=Test['fav']
Test['Vote_favor'][Test['Voter_preds']>Test['Open']]=Test['dog']


#TELLS US WHETHER OR NOT OUR MODEL FAVORED THE TEAM THAT COVERED THE SPREAD
Test['result_Vote']=np.NaN
Test['result_Vote'][Test['Vote_favor']==Test['spread_result']]=1
Test['result_Vote'][Test['Vote_favor']!=Test['spread_result']]=0
#Test['result_gbr']=np.NaN
#Test['result_gbr'][Test['Vote_favor']==Test['spread_result']]=1
#Test['result_gbr'][Test['Vote_favor']!=Test['spread_result']]=0
#Test['result_xgb']=np.NaN
#Test['result_xgb'][Test['Vote_favor']==Test['spread_result']]=1
#Test['result_xgb'][Test['Vote_favor']!=Test['spread_result']]=0


#BET IF OUR LINE IS AT LEAST 3.5 POINTS DIFFERENT THAN THE OPENING LINE
Bets=Test[Test['Vote_guess']>=3.5]

print('Voter % of games correctly predicted',sum(Bets['result_Vote'])/(len(Bets)))
print('Bets Made: ',len(Bets))
print()
print()


Bets.head(51)


Voter % of games correctly predicted 0.5740740740740741
Bets Made:  54




,Date,schedule_week,score_home,score_away,home,away,fav,dog,home_favorite_dup,away_favorite_dup,Open,Close,spread_result,Pick,Voter_preds,gbr_preds,xgb_preds,Vote_guess,Vote_favor,result_Vote
2937,2018-09-06,1,18.0,12.0,PHI,ATL,ATL,PHI,0.0,1.0,4.0,-0.0,PHI,NaN,-4.469854,-3.521247,-3.930997,8.469854,ATL,0.0
2943,2018-09-09,1,24.0,23.0,GB,CHI,GB,CHI,1.0,0.0,-8.5,-7.0,CHI,NaN,-3.912460,-4.016868,-3.878477,4.587540,CHI,1.0
2944,2018-09-09,1,23.0,34.0,IND,CIN,CIN,IND,1.0,0.0,1.0,-1.0,CIN,NaN,-2.858560,-2.724631,-2.825824,3.858560,CIN,1.0
2946,2018-09-09,1,27.0,20.0,MIA,TEN,MIA,TEN,1.0,0.0,2.5,-0.0,MIA,NaN,-2.653598,-2.041432,-2.348248,5.153598,MIA,1.0
2948,2018-09-09,1,27.0,20.0,NE,HOU,NE,HOU,1.0,0.0,-6.5,-6.0,NE,NaN,-11.856188,-12.696983,-13.111802,5.356188,NE,1.0
2955,2018-09-16,2,20.0,31.0,BUF,LAC,LAC,BUF,0.0,1.0,-7.0,-7.5,LAC,NaN,-3.254403,-4.315186,-3.809765,3.745597,BUF,0.0
2958,2018-09-16,2,29.0,29.0,GB,MIN,MIN,GB,1.0,0.0,2.0,-1.5,GB,NaN,-2.807050,-2.238052,-2.546928,4.807050,MIN,0.0
2960,2018-09-16,2,34.0,0.0,LAR,ARI,LAR,ARI,1.0,0.0,-13.0,-12.0,LAR,NaN,-8.778193,-8.770203,-9.558520,4.221807,ARI,0.0
2967,2018-09-16,2,9.0,21.0,WAS,IND,WAS,IND,1.0,0.0,-4.5,-6.0,IND,NaN,-8.414129,-9.254039,-8.622604,3.914129,WAS,0.0
2980,2018-09-23,3,6.0,27.0,MIN,BUF,MIN,BUF,1.0,0.0,-16.0,-17.0,BUF,NaN,-11.477025,-11.182310,-11.078029,4.522975,BUF,1.0


In [24]:
#RUNNING THIS CELL WILL TELL YOU THE NUMBER OF BETS YOU MADE ON HOME FAVORITES AND % YOU PICKED CORRECTLY
#DOES THE SAME WITH AWAY FAVS, HOME DOGS, AND AWAY DOGS AND PICK 'EM GAMES (NO FAVORITE) 

#THIS TELLS YOU WHICH AREAS THE MODEL PERFORMS WELL IN AND WHICH AREAS IT DOES NOT.
#YOU CAN THEN TRY MAKING ADJUSTMENTS TO THE MODEL TO IMPROVE ITS PERFORMANCE IN ALL AREAS.


no_push=Test[Test['score_home']-Test['score_away']!=Test['Open']]
no_push=no_push[no_push['score_away']-no_push['score_home']!=no_push['Open']]
no_push_copy=no_push.copy()
no_push=no_push[(no_push['home_favorite_dup']==1)|(no_push['away_favorite_dup']==1)]

home_fav_cov=no_push[(no_push['home_favorite_dup']==1)&(no_push['fav']==no_push['spread_result'])]
home_fav_bets=home_fav_cov[home_fav_cov['Vote_guess']>=3.5]
print('Voter % of home favorites correctly predicted',sum(home_fav_bets['result_Vote'])/(len(home_fav_bets)))
print('Bets Made: ',len(home_fav_bets))
print()

home_dog_cov=no_push[(no_push['home_favorite_dup']==0)&(no_push['dog']==no_push['spread_result'])]
home_dog_bets=home_dog_cov[home_dog_cov['Vote_guess']>=3.5]
print('Voter % of home dogs correctly predicted',sum(home_dog_bets['result_Vote'])/(len(home_dog_bets)))
print('Bets Made: ',len(home_dog_bets))
print()

away_fav_cov=no_push[(no_push['away_favorite_dup']==1)&(no_push['fav']==no_push['spread_result'])]
away_fav_bets=away_fav_cov[away_fav_cov['Vote_guess']>=3.5]
print('Voter % of away favorites correctly predicted',sum(away_fav_bets['result_Vote'])/(len(away_fav_bets)))
print('Bets Made: ',len(away_fav_bets))
print()

away_dog_cov=no_push[(no_push['away_favorite_dup']==0)&(no_push['dog']==no_push['spread_result'])]
away_dog_bets=away_dog_cov[away_dog_cov['Vote_guess']>=3.5]
print('Voter % of away dogs correctly predicted',sum(away_dog_bets['result_Vote'])/(len(away_dog_bets)))
print('Bets Made: ',len(away_dog_bets))
print()

pickems=no_push_copy[no_push_copy['Open']==0]
pickems_bets=pickems[pickems['Vote_guess']>=3.5]
pickems_home=pickems_bets[pickems_bets['spread_result']==pickems_bets['home']]
pickems_away=pickems_bets[pickems_bets['spread_result']==pickems_bets['away']]
print('home pick em correct prediction %: ',sum(pickems_home['result_Vote'])/(len(pickems_home)))
print('away pick em correct prediction %: ',sum(pickems_away['result_Vote'])/(len(pickems_away)))
print('Bets made on pick em: ',len(pickems_bets))



Voter % of home favorites correctly predicted 0.6
Bets Made:  15

Voter % of home dogs correctly predicted 0.2857142857142857
Bets Made:  7

Voter % of away favorites correctly predicted 0.4444444444444444
Bets Made:  9

Voter % of away dogs correctly predicted 0.75
Bets Made:  20

home pick em correct prediction %:  0.0
away pick em correct prediction %:  1.0
Bets made on pick em:  2
